In [1]:
import cv2 as cv
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from matplotlib import pyplot as plt
import time
import sys
import multiprocessing
import os
import rospy
from sensor_msgs.msg import Image
from stereo_msgs.msg import DisparityImage
from sensor_msgs.msg import PointCloud2
from nav_msgs.msg import OccupancyGrid as Map
import message_filters
import cv_bridge
import tf
import math
from std_msgs.msg import String

In [2]:
def createLineIterator(P1, P2, img):
# """
# Produces and array that consists of the coordinates and intensities of
# each pixel in a line between two points

# Parameters:
#     -P1: a numpy array that consists of the coordinate of the first point (x,y)
#     -P2: a numpy array that consists of the coordinate of the second point (x,y)
#     -img: the image being processed

# Returns:
#     -it: a numpy array that consists of the coordinates and intensities of each
#     pixel in the radii (shape: [numPixels, 3], row = [x,y,intensity])     
# """
   #define local variables for readability
    #print img.shape
    image_height = img.shape[0]
    image_width = img.shape[1]
    P1X = P1[0]
    P1Y = P1[1]
    P2X = P2[0]
    P2Y = P2[1]
    
   #difference and absolute difference between points
   #used to calculate slope and relative location between points
    dX = P2X - P1X
    dY = P2Y - P1Y
    dXa = np.abs(dX)
    dYa = np.abs(dY)

    #predefine numpy array for output based on distance between points
    itbuffer = np.empty(shape=(np.maximum(dYa,dXa),3), dtype=np.int32)
    itbuffer.fill(np.nan)

   #Obtain coordinates along the line using a form of Bresenham's algorithm
    negY = P1Y > P2Y
    negX = P1X > P2X
    if P1X == P2X: #vertical line segment
        itbuffer[:,0] = P1X
        if negY:
            itbuffer[:,1] = np.arange(P1Y - 1,P1Y - dYa - 1,-1)
        else:
            itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)              
    elif P1Y == P2Y: #horizontal line segment
        itbuffer[:,1] = P1Y
        if negX:
            itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
        else:
            itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
    else: #diagonal line segment
        steepSlope = dYa > dXa
        if steepSlope:
            slope = dX.astype(np.float32)/dY.astype(np.float32)
            if negY:
                itbuffer[:,1] = np.arange(P1Y-1,P1Y-dYa-1,-1)
            else:
                itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)
            itbuffer[:,0] = (slope*(itbuffer[:,1]-P1Y)).astype(np.int) + P1X
        else:
            slope = dY.astype(np.float32)/dX.astype(np.float32)
            if negX:
                itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
            else:
                itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
            itbuffer[:,1] = (slope*(itbuffer[:,0]-P1X)).astype(np.int) + P1Y

   #Remove points outside of image
    colX = itbuffer[:,0]
    colY = itbuffer[:,1]
    itbuffer = itbuffer[(colX >= 0) & (colY >=0) & (colX<image_width) & (colY<image_height)]

   #Get intensities from img ndarray
    itbuffer[:,2] = img[itbuffer[:,1].astype(np.uint),itbuffer[:,0].astype(np.uint)]
    return itbuffer

In [3]:
# Initialization Stuff

rospy.init_node('liveproc')

listener = tf.TransformListener()
pub = rospy.Publisher('updated_map', Map, queue_size=10)

occ_map = None
up_occ_map = None
map_resolution = 0.5
origin = None
robot_x = 0
robot_y = 0
map_width = 0
map_height = 0
camera_range = 85/map_resolution
overlay = False
display_robot_yaw = False
display_rect = True
display_map = True
display_info = False
tri_width = 10

np.set_printoptions(threshold=sys.maxsize)

In [4]:
def edgeFunction(a,b,c):
    det = (c[1]-a[1]) * (b[0]-a[0]) - (c[0]-a[0]) * (b[1]-a[1])
    return det

In [5]:
def rasterizeTri(a,b,c,d,region):
    a[0] = a[0] + (d[0]-c[0])
    a[1] = a[1] + (d[1]-c[1])
    b[0] = b[0] + (d[0]-c[0])
    b[1] = b[1] + (d[1]-c[1])
    c[0] = d[0]
    c[1] = d[1]
    new_region = np.zeros_like(region)
    for i in range(len(region)):
        for j in range(len(region[i])):
            area = edgeFunction(a,b,c)
            w0 = edgeFunction(b,c,(i,j))
            w1 = edgeFunction(c,a,(i,j))
            w2 = edgeFunction(a,b,(i,j))

            if w0 >= 0 and w1 >= 0 and w2 >= 0:
                if region[i][j] == values.get("green"):
                    new_region[i][j] = 255
                elif region[i][j] == values.get("red"):
                    new_region[i][j] = 180
                else:
                    new_region[i][j] = 100
    return new_region


In [6]:
def rectangle(mat,color,image,original):
    
    global bearings
    global overlay
    global occ_map
    global tags
    global display_info
    global map_resolution
    global areas
    
    rect = cv.boundingRect(mat)
    min_rect = cv.minAreaRect(mat)

    height = rect[3]
    width = rect[2]
    x = rect[0]
    x = np.int0(x)
    y = rect[1] 
    y = np.int0(y)
    font = cv.FONT_HERSHEY_SIMPLEX
    tag = 0
    if overlay:
        if (width < height and 4*width > height and width*height > 45):
            if(color =='green'):
                tag = 2
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,128,0),2,cv.LINE_AA)
                tags.append(tag)
            elif(color =='red'):
                tag = 3
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,0,128),2,cv.LINE_AA)
                tags.append(tag)
            else:
                tag = 1
                cv.putText(original,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            cv.rectangle(original,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = (min_rect[0][0]-640.0)/1280.0*80.0
            bearings.append(bearing)
            area = min_rect[1][0]*min_rect[1][1]
            areas.append(area)
#             print("Area: " + str(area) + " and Tag: " + str(tag) +
#                   "\nEstimated Distance: " + str(2*map_resolution*456.7946397925*math.pow(area,-0.4705300238)))
            if display_info and occ_map.header.seq % 5 == 0:
                print bearings
                print tags
        if(tag == 0):
            image[y:y+height][x:x+width]=[0,0,0]
        return original
    else:
        if (width < height and 4*width > height and width*height > 45):
            if(color =='green'):
                tag = 2
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            elif(color =='red'):
                tag = 3
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            else:
                tag = 1
                cv.putText(image,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            cv.rectangle(image,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = (min_rect[0][0]-640.0)/1280.0*80.0
            bearings.append(bearing)
            area = min_rect[1][0]*min_rect[1][1]
            areas.append(area)
#             print("Area: " + str(area) + " and Tag: " + str(tag) +
#                   "\nEstimated Distance: " + str(2*map_resolution*456.7946397925*math.pow(area,-0.4705300238)))
            if display_info and occ_map.header.seq % 5 == 0:
                print bearings
                print tags
        return image

In [7]:
def get_mapnbot_info(trans,robot_yaw):

    global map_resolution
    global robot_x
    global robot_y
    global map_width
    global map_height
    global origin
    global occ_map
    
    map_array = occ_map.data

    map_width = occ_map.info.width
    map_height = occ_map.info.height
    map_np = np.reshape(np.array(map_array),(map_height,map_width))
    robot_x = (trans[0]-origin.x)/map_resolution
    robot_y = (trans[1]-origin.y)/map_resolution
    robot_yaw = (90 - robot_yaw)
    while robot_yaw > 360:
        robot_yaw = robot_yaw - 360
    while robot_yaw < 0:
        robot_yaw = robot_yaw + 360
    
    return map_np, robot_yaw

In [8]:
def disp_rect(rect):
    cv.namedWindow("stereo/left/image_rect_color",cv.WINDOW_NORMAL)
    cv.imshow("stereo/left/image_rect_color", rect)
    cv.waitKey(1)

In [9]:
def get_points(buoy_yaw):
    
    global robot_x
    global robot_y
    global camera_range
    global tri_width
    
    point_deltax = camera_range * math.sin(math.radians(buoy_yaw))
    point_deltay = camera_range * math.cos(math.radians(buoy_yaw))
    per_slope = -1.0/(point_deltay/point_deltax)
    point1 = np.array([round(robot_x), round(robot_y)])
    point2 = np.array([round(robot_x+point_deltax), round(robot_y+point_deltay)])
    
    point3_x = (robot_x+point_deltax) + math.sqrt(math.pow(tri_width,2)/(math.pow(per_slope,2)+1))
    point4_x = (robot_x+point_deltax) - math.sqrt(math.pow(tri_width,2)/(math.pow(per_slope,2)+1))
#     print "outside"
    point3_y = robot_y+point_deltay + per_slope*(point3_x-(robot_x+point_deltax))
    point4_y = robot_y+point_deltay + per_slope*(point4_x-(robot_x+point_deltax))
#     print "outside"
    point3 = np.array([round(point3_x),round(point3_y)])
    point4 = np.array([round(point4_x),round(point4_y)])
    
    
    return point1,point2,point3,point4

In [10]:
def map_filter(trans, robot_yaw):
    
    global origin
    global robot_x
    global robot_y
    global camera_range
    global occ_map
    global bearings
    global display_robot_yaw
    global tags
    
    map_np, robot_yaw = get_mapnbot_info(trans,robot_yaw) 
    
##############################################################################
# This is for visualization purposes.
# Set to false if you don't want to display the robot_yaw
# Delete it if it's irritating to look at
    if display_robot_yaw and occ_map.header.seq % 5 == 0:
        yaw_offset.append(0.0)
        print "Robot_yaw: ", robot_yaw
        print "Buoy yaw: ", buoy_yaw
        print len(slices)
##############################################################################
    
    slices = []
    i = 0
    while (i < len(bearings)):
        
        
        buoy_yaw = robot_yaw + bearings[i]
        if ((buoy_yaw) < 0):
            buoy_yaw = buoy_yaw + 360
        
        
        
        pt1, pt2, pt3, pt4 = get_points(buoy_yaw)
        
        
        sliced = createLineIterator(pt1.astype(int),pt2.astype(int),map_np)
        slices.append(sliced)
###############################################################################################     

# If you uncomment the code below, make sure you change tags[i//1] to tags[i//3] in the draw_ray function

        
        sliced = createLineIterator(pt1.astype(int),pt3.astype(int),map_np)
        slices.append(sliced)
        
        sliced = createLineIterator(pt1.astype(int),pt4.astype(int),map_np)
        slices.append(sliced)
################################################################################################
        i+=1
    
    return slices

In [11]:
def draw_background_objects(map_img,map_np):
    
    global origin
    global robot_x
    global robot_y
    
    o_x = (-origin.x)/map_resolution
    o_y = (-origin.y)/map_resolution
# Color Assignment Section
    map_img[map_np == -1] = [255,255,255]    #Change to [255,255,255] for color to [255] for grayscale
    map_img[map_np == 0] = [255,255,255]     #Change to [255,255,255] for color to [255] for grayscale
    map_img[map_np == 100] = [0,0,0]#Change to [0,0,0] for color to [255] for grayscale
    origin_color = [0,0,255]#Change to [0,0,255] for color to [255] for grayscale
    robot_color = [0,0,255]#Change to [0,0,255] for color to [255] for grayscale
# Origin (Drawn as a cross)

    map_img[int(o_y)+0][int(o_x)+0] = origin_color
    map_img[int(o_y)+0][int(o_x)+1] = origin_color
    map_img[int(o_y)+0][int(o_x)+2] = origin_color
    map_img[int(o_y)+1][int(o_x)+0] = origin_color
    map_img[int(o_y)+2][int(o_x)+0] = origin_color
    if (-origin.x) != 0:
        map_img[int(o_y)+0][int(o_x)-1] = origin_color
        map_img[int(o_y)+0][int(o_x)-2] = origin_color
    if (-origin.y) != 0:
        map_img[int(o_y)-1][int(o_x)+0] = origin_color
        map_img[int(o_y)-2][int(o_x)+0] = origin_color
# Robot (Drawn as a square)
    map_img[int(robot_y)+0][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+0][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+0][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)+2][int(robot_x)-1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)-1] = robot_color
    
    return map_img

In [12]:
def draw_ray(map_img,slices):
    
    global display_robot_yaw
    global bearings
    global tags
    global areas
    global robot_x
    global robot_y
    global map_resolution
    
    slices_size = len(slices)
    
    if not(display_robot_yaw):
        for i in range(int(slices_size)):
            try:
                found = False
                arr = slices[i]
                for ray in arr:
                    #print slice
                    if not found and (ray[2] == -1 or ray[2] == 0):
                        map_img[ray[1]][ray[0]] = [255,0,0]#Change to [255,0,0] for color to [0] for grayscale
                    elif ray[2] == 100:
                        if tags[i//1] == 2: #here
                            map_img[ray[1]][ray[0]]=[0,255,0]#Change to [0,255,0] for color to [180 for grayscale]
                            print("Distance from buoy: " + str(map_resolution*math.sqrt(math.pow((ray[1]-robot_y),2)+math.pow((ray[0]-robot_x),2))) 
                                  + "Tag: green"
                                  + "Area: " + str(areas[i]) +
                                  "\nEstimated Distance: " + str(2*map_resolution*456.7946397925*math.pow(areas[i],-0.4705300238)))
                            print ("")
                            found = True
                        if tags[i//1] == 3: #and here
                            map_img[ray[1]][ray[0]]=[0,0,255]#Change to [0,0,255] for color to [230] for grayscale
                            print("Distance from buoy: " + str(map_resolution*math.sqrt(math.pow((ray[1]-robot_y),2)+math.pow((ray[0]-robot_x),2))) 
                                  + " Tag: red"
                                  + "Area: " + str(areas[i]) +
                                  "\nEstimated Distance: " + str(2*map_resolution*456.7946397925*math.pow(areas[i],-0.4705300238)))
                            print ("")
                            found = True
            except Exception as ex:
                print ex
    else:
        for arr in slices[-1]:
            map_img[arr[1]][arr[0]] = [0,255,0]
    
    return map_img

In [13]:
def get_map(slices):
    
    global camera_range
    global origin
    global robot_x
    global robot_y
    global map_width
    global map_height
    global occ_map
    
    map_np = np.reshape(np.array(occ_map.data),(map_height,map_width))

    map_img = np.zeros((map_height, map_width, 3))
    
    map_img = draw_background_objects(map_img,map_np)
    
    map_img = draw_ray(map_img,slices)
    
    map_img = np.swapaxes(map_img, 0, 1)
    
    return map_img

In [14]:
def disp_map(map_img):
    cv.namedWindow("map_inverted",cv.WINDOW_NORMAL)
    cv.imshow("map_inverted", map_img)
    cv.waitKey(1)

In [15]:
def transform(img):
    
    global bearings
    global areas
    global tags
# Once a color image has been read, it passes through the 
# transform function. The transform function has two parts:
# pre-processing also known as pp which filters the image based
# of certain values in the hue, saturation, and value (hsv) 
# colorspace and the dbscan which clusters the processed image
# and classifies the clusters as either a buoy, a green buoy,
# a red buoy, or noise. No changes are needed.
    
    
    
    bridge = cv_bridge.CvBridge()
    
    areas = []
    bearings = []
    tags = []
    
    width = img.height
    height = img.width
    
    img = bridge.imgmsg_to_cv2(img, desired_encoding="passthrough")
    
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)  
    
    
    image = pp(img,width,height)
    
    img2= dbscan(image, img)
    
    return img2
def pp(image, width, height):
    
# The first function in transform is pp or preprocessing. The 
# first line truncates the lower half of the image. The reason
# this is because the plantoons from the wamv are always 
# blocking the camera vision and become noisy data. The
# function changes the image to the HSV colorspace and 
# filters out the image. No changes are needed. It returns 
# the filtered image in the BGR colorspace.The input is the
# color image.

    image = np.array(image[0:width][0:int(height*0.65)])
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_and(image[:,:,1] > 5,np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10))))] = [0,0,0]
    image = cv.cvtColor(image,cv.COLOR_HSV2BGR)

    return(image)
def dbscan(image, original):
    start_time_db = time.time()
# The second function in transform is dbscan. The inputs are 
# the filtered colored image and the original color image
# read from the imread statement. The output is the original
# image.This is the end of this classification algorithm.

###########################################################

# Part 1: Thresholding the image
##########################################################
# The reason we threshold the image before applying DBSCAN to 
# the points is because DBSCAN is a computational demanding 
# function. This part takes the filtered image, makes a 
# copy of it, and converts it into a grayscale image. The gray-
# scale image is thresholded at a value of 10. 0 is black and
# 255 is white. This takes the ~400,000 matrix points and 
# leaves only ~10,000 nonzero matrix points. 

    img = image.copy()
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret,thresh1 = cv.threshold(img,10,255,cv.THRESH_BINARY)

# After thresholding the image, we pull out those nonzero
# points and assign it to the nonzero array variable. This
# array has dimensions of n rows and 2 columns. The first 
# column is the y coordinate and the second column is the x
# coordinate. These two arrays would be combined by 
# np.column_stack which merges the two arrays at their columns.

    nonzero = np.nonzero(thresh1)

    yp = np.array(nonzero[0])
    xp = np.array(nonzero[1])


    X=np.column_stack((xp,yp))
#########################################################

# Creating the BDSCAN object and Sorting Array with Labels
#########################################################    
# After thresholding the filtered image and creating an
# array of nonzero coordinates, this algorithm applies a
# clustering algorithm that filters the image based off the
# array X. If X is empty, this function ends and output is 
# the original input. The first line creates the DBSCAN
# object with an epsilon of 3 and minimum samples of 20. 
# Eps refers to the size of the neighborhood and the 
# min_samples refers to the minimum number of samples required
# for that neighborhood to be considered a cluster. The user
# is free to change the two values for eps and min_samples. The
# .fit(X) applies the matrix points to the DBSCAN object.
# One of the parameters of the db scan object are the labels.
# This is invoked by the line labels = db.labels. This creates
# an array of labels that assigns a number to the image index.
# n clusters have labels that start at 0 and end with n-1.For 
# example, if an image has four clusters, the labels would
# include 0,1,2, and 3. -1 is dedicated for noise. 

    if (len(X) > 0):
        start_time = time.time()
        db = DBSCAN(eps=3, min_samples=10).fit(X)
        labels = db.labels_
#         print (1.0*(time.time()-start_time))

# The array variable combines the x and y coordinates and the 
# labels in a single array aligned by column. The array is
# sorted with the sort variable with the values in the third
# column or the labels. For example, an image with four
# clusters and noise would be labeled in order from (-1,0,1,2,3).
# The while loop iterates from the beginning of the sorted loop
# and deletes every instance of -1 or until the list is empty.
# This while loop is needed because the noise data is
# irrelevant to the cluster algorithm and the user does not 
# know how much noise there is in the image. The image could 
# have no noise or it could all be noise. The output of this
# section is an array with coordinates sorted together in 
# clusters.

        array = zip(xp,yp,labels)
        sort = sorted(list(array), key=lambda x: x[2])

        x = 0
        while (len(sort) > 0 and sort[x][2] == -1):
            del sort[x]
#########################################################

# Iterating through and classifying each cluster
#########################################################
# After iterating through sort, the program is either left with
# an empty array or an array of coordinates sorted and grouped
# by each cluster. The user needs the number of clusters. The 
# next three lines in the big if block does this. unique_labels
# sorts the set of labels and removes -1 if present. The next 
# two lines creates a copy of the sort array and removes the 
# labels column from the array.

        if (len(sort) > 0):
            unique_labels = sorted(set(labels))
            if (unique_labels[0] == -1):
                unique_labels.remove(-1)
            points = sort
            points = np.delete(points,2,1)

# The program assigns three variables. The first one i iterates
# through sort and compare its label value to x for each 
# iteration until the very end of the sort array. The second
# x refers to the cluster that the program is currently class-
# ifying. It starts with 0 and continues classifying until it 
# has classified all clusters. The third variable is l and this
# array becomes all the points within a cluster. Every time the 
# variable i iterates through sort and is equal to the value of
# the current cluster x, the coordinates are appended to the 
# array l. Since this array is sorted, the first time the x 
# variable equals a different value- the cluster has been 
# accounted for. This array of l contains all the points within
# the cluster. This array and the filtered image gets passed
# through the color function. The result of the color function
# gets passed through the rectangle function with the array,
# filtered image, and original image. The x variable is 
# incremented for the next cluster and the l array is set equal 
# to zero. Once the while loop has iterated through all 
# clusters, the result of the rectangle function is set to the 
# original image and the original image is returned to the 
# transform function which is returned to the read function 
# which writes the image.

            i = 0
            x = 0
            l = []
            while (x < len(unique_labels)):
                if (i < len(sort) and sort[i][2] == x):
                    l.append(points[i])
                    i+=1
                else:
                    x+=1
#                     time_color = time.time()
                    clr = color_hsv(np.array(l),image)
#                     print "color ", time.time()-time_color
#                     time_rect = time.time()
                    tb = rectangle(np.array(l),clr,image,original)
#                     print "rect ", time.time()-time_rect
#                     print ""
                    l = []
            original = tb
#     print time.time()-start_time_db
#     print "\n"
    return original
def color(mat,image):
    
    ############################################################

# For the color function, the points of the cluster and the 
# filtered image are passed through. In this function, the 
# average red, green, and blue pixel values are calculated
# from the image inside of the cluster. If the average value
# fits within a certain range, the string 'red' or 'green' is 
# returned. Otherwise, the function returns the string 'none'
# The user is free to change the range of the BGR values that
# determine whether the buoy is red or green.

    sumred = 0
    sumgrn = 0
    sumblu = 0
    for x in range(len(mat)):
        sumred += image [mat[x][1]] [mat[x][0]] [2]
        sumgrn += image [mat[x][1]] [mat[x][0]] [1]
        sumblu += image [mat[x][1]] [mat[x][0]] [0]
    if ((sumred/len(mat)) > 80 and (sumred/len(mat)) < 180 and (sumgrn/len(mat)) > 30 and (sumgrn/len(mat)) < 115  and (sumblu/len(mat)) > 30 and (sumblu/len(mat)) < 110):
        return 'red'
    if ((sumred/len(mat)) > 30 and (sumred/len(mat)) < 80 and (sumgrn/len(mat)) > 60 and (sumgrn/len(mat)) < 180  and (sumblu/len(mat)) > 50 and (sumblu/len(mat)) < 130):
        return 'green'
    else:
        return 'none'
def color_hsv(mat, image):
    
# The rectangle function's parameters are the array of coordinates
# of the cluster, the color of the cluster, the filtered image,
# and the original image. The boundingRect function applies
# a non-rotated rectangle to the image points and returns four
# values in an array, the x and y coordinates of the leftmost
# corner and the width and height. These values are converted
# to ints. The font is set to Hershey Simple for the text and
# the tag is set to zero. The tag is useful in the classifica-
# tion of the buoy. The first classification of the buoy is 
# the proportion of the width and height. If the height is
# between 1.5 times the width and 3.5 times the width, the
# cluster is considered a buoy. If the cluster is given a 
# 'red' or 'green' color, it would be classified as a 'red buoy'
# or 'green buoy' respectively. Otherwise, it would simply be
# classified as a buoy. If any of these three options occur, 
# the tag is set to 1. If the height is not in that range
# mentioned above and the color is 'none', the cluster is
# classified as noise. The last function rectangle makes a 
# rectangle over the original image. The original image is sent
# back to the DBSCAN function.


    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    sumhue = 0
    sumsat = 0
    sumval = 0
    mat_add = 0
    for x in range(len(mat)):
        if(image [mat[x][1]] [mat[x][0]] [2] > 5):
            mat_add = mat_add+1
            sumval += image [mat[x][1]] [mat[x][0]] [2]
            sumsat += image [mat[x][1]] [mat[x][0]] [1]
            sumhue += image [mat[x][1]] [mat[x][0]] [0]
    if mat_add > 0:
        sat = sumsat/mat_add
        val = sumval/mat_add
        hue = sumhue/mat_add
        #return (hue, sat, val)
        if (sat > 5 and val > 5 and (hue < 60 or hue >= 170)):
            return 'red'
        if (sat > 5 and val > 5 and (70 < hue < 90)):
            return 'green'
        else:
            return (hue, sat, val)

    return None

In [16]:
def callback(rect):
    
    global occ_map
    global origin
    global bearings
    global listener 
    global display_rect
    global display_map
    global tags
    global up_occ_map
    global pub
    
    origin = occ_map.info.origin.position
    rect = transform(rect)
    
    if not rospy.is_shutdown():       
        try:
            now = rospy.Time.now()
            listener.waitForTransform("/odom", "/front_left_camera_link", now, rospy.Duration(0.5))
            (trans,rot) = listener.lookupTransform("/odom", "/front_left_camera_link", now)
            euler = tf.transformations.euler_from_quaternion(rot)
            pixel_lines = map_filter(trans, math.degrees(euler[2]))
            map_image = get_map(pixel_lines)
#             print(map_image)
#             up_occ_map.data = map_image
            pub.publish(up_occ_map)
            if display_rect == True:
                disp_rect(rect)
            if display_map == True:
                disp_map(map_image)
        except tf.LookupException as l:
            print l
        except tf.ConnectivityException as c:
            print c
        except tf.ExtrapolationException as e:
            print e
        except Exception as ex:
            print ex 

In [17]:
def map_callback(p_map):
    
    global occ_map
    global up_occ_map
    
    occ_map = p_map    
    up_occ_map = p_map

In [ ]:
if __name__ == '__main__':
    proj_map = rospy.Subscriber("projected_map", Map, map_callback)
    rect = rospy.Subscriber("stereo/left/image_rect_color", Image, callback)
    rospy.spin()   

Distance from buoy: 35.2888605414 Tag: redArea: 138.0
Estimated Distance: 44.9617950498

Distance from buoy: 35.7835605224 Tag: redArea: 138.0
Estimated Distance: 44.9617950498

Distance from buoy: 35.290988902 Tag: redArea: 138.0
Estimated Distance: 44.9617950498

Distance from buoy: 35.785690671 Tag: redArea: 138.0
Estimated Distance: 44.9617950498

Distance from buoy: 35.2928419716 Tag: redArea: 138.0
Estimated Distance: 44.9617950498

Distance from buoy: 35.7875452798 Tag: redArea: 138.0
Estimated Distance: 44.9617950498

Distance from buoy: 35.2933308615 Tag: redArea: 159.208235913
Estimated Distance: 42.0368442599

Distance from buoy: 35.7880351256 Tag: redArea: 159.208235913
Estimated Distance: 42.0368442599

Distance from buoy: 35.295015473 Tag: redArea: 161.0
Estimated Distance: 41.8160654761

Distance from buoy: 35.7897208251 Tag: redArea: 161.0
Estimated Distance: 41.8160654761

Distance from buoy: 35.2962947514 Tag: redArea: 216.0
Estimated Distance: 36.4158473596

Distance

Distance from buoy: 35.3222506645 Tag: redArea: 216.0
Estimated Distance: 36.4158473596

Distance from buoy: 35.8169907895 Tag: redArea: 216.0
Estimated Distance: 36.4158473596

Distance from buoy: 35.3239685995 Tag: redArea: 207.0
Estimated Distance: 37.1524464876

Distance from buoy: 35.818710123 Tag: redArea: 207.0
Estimated Distance: 37.1524464876

Distance from buoy: 35.3252789783 Tag: redArea: 165.856608453
Estimated Distance: 41.2353875947

Distance from buoy: 35.8200214792 Tag: redArea: 165.856608453
Estimated Distance: 41.2353875947

Distance from buoy: 35.3264735716 Tag: redArea: 154.0
Estimated Distance: 42.6998964907

Distance from buoy: 35.8212169189 Tag: redArea: 154.0
Estimated Distance: 42.6998964907

Distance from buoy: 35.3293998726 Tag: redArea: 147.0
Estimated Distance: 43.6448617329

Distance from buoy: 35.8241455411 Tag: redArea: 147.0
Estimated Distance: 43.6448617329

Distance from buoy: 35.3269243317 Tag: redArea: 126.0
Estimated Distance: 46.9281675803

Distan

Distance from buoy: 35.3614177404 Tag: redArea: 260.0
Estimated Distance: 33.3736365627

Distance from buoy: 35.8562000562 Tag: redArea: 260.0
Estimated Distance: 33.3736365627

Distance from buoy: 35.3625994672 Tag: redArea: 260.0
Estimated Distance: 33.3736365627

Distance from buoy: 35.8573831214 Tag: redArea: 260.0
Estimated Distance: 33.3736365627

Distance from buoy: 35.3623348667 Tag: redArea: 260.0
Estimated Distance: 33.3736365627

Distance from buoy: 35.8571186052 Tag: redArea: 260.0
Estimated Distance: 33.3736365627

Distance from buoy: 35.3625354507 Tag: redArea: 263.377304446
Estimated Distance: 33.1715840626

Distance from buoy: 35.8573196339 Tag: redArea: 263.377304446
Estimated Distance: 33.1715840626

Distance from buoy: 35.362259642 Tag: redArea: 250.0
Estimated Distance: 33.9952492505

Distance from buoy: 35.8570436507 Tag: redArea: 250.0
Estimated Distance: 33.9952492505

Distance from buoy: 35.3622654771 Tag: redArea: 256.353752605
Estimated Distance: 33.5961574168

Distance from buoy: 35.3980675713 Tag: redArea: 207.0
Estimated Distance: 37.1524464876

Distance from buoy: 35.8928872832 Tag: redArea: 207.0
Estimated Distance: 37.1524464876

Distance from buoy: 35.3992334145 Tag: redArea: 207.0
Estimated Distance: 37.1524464876

Distance from buoy: 35.8940548795 Tag: redArea: 207.0
Estimated Distance: 37.1524464876

Distance from buoy: 35.3996358473 Tag: redArea: 224.224141637
Estimated Distance: 35.7811560069

Distance from buoy: 35.8944580798 Tag: redArea: 224.224141637
Estimated Distance: 35.7811560069

Distance from buoy: 35.3999881761 Tag: redArea: 225.0
Estimated Distance: 35.7230476482

Distance from buoy: 35.8948111519 Tag: redArea: 225.0
Estimated Distance: 35.7230476482

Distance from buoy: 35.403042887 Tag: redArea: 225.0
Estimated Distance: 35.7230476482

Distance from buoy: 35.8978685582 Tag: redArea: 225.0
Estimated Distance: 35.7230476482

Distance from buoy: 35.4020143459 Tag: redArea: 234.0
Estimated Distance: 35.0698416811

Distan

Distance from buoy: 35.4337213316 Tag: redArea: 207.0
Estimated Distance: 37.1524464876

Distance from buoy: 35.9285813942 Tag: redArea: 207.0
Estimated Distance: 37.1524464876

Distance from buoy: 35.4341610178 Tag: redArea: 184.0
Estimated Distance: 39.2695762509

Distance from buoy: 35.9290217426 Tag: redArea: 184.0
Estimated Distance: 39.2695762509

Distance from buoy: 35.4352219904 Tag: redArea: 207.0
Estimated Distance: 37.1524464876

Distance from buoy: 35.9300835143 Tag: redArea: 207.0
Estimated Distance: 37.1524464876

Distance from buoy: 35.4360120899 Tag: redArea: 186.777372488
Estimated Distance: 38.993726407

Distance from buoy: 35.9308744428 Tag: redArea: 186.777372488
Estimated Distance: 38.993726407

Distance from buoy: 35.4366386676 Tag: redArea: 160.769238647
Estimated Distance: 41.844296439

Distance from buoy: 35.9315017331 Tag: redArea: 160.769238647
Estimated Distance: 41.844296439

Distance from buoy: 35.4362399799 Tag: redArea: 184.0
Estimated Distance: 39.26957